In [1]:
import sys
import re
import time
import urllib

import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [2]:
url = 'https://paperswithcode.com/datasets?mod=images&page=1'
res = requests.get(url)

soup = BeautifulSoup(res.text, 'html.parser')
title = soup.find_all(class_='filter-items')

counter = 0
title_list = []
while True:
    try:
        text = title[1].contents[counter].get_text(strip=True)
        if text != '':
            text = re.sub(r'\d*$', '', text)
            title_list.append(text)
        counter += 1
    except IndexError:
        break

len(title_list)

499

In [3]:
# browser = webdriver.Chrome()

# ヘッドレスモード
from selenium.webdriver.chrome.options import Options
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(options=options)


browser.set_window_size('1200', '1000')
browser.get('https://paperswithcode.com/')

obj = []
not_page =[]

In [4]:
for i in range(len(title_list)):

    task_obj = {}
    task_obj['task'] = title_list[i]

    box = browser.find_element(By.NAME, 'q')
    box.send_keys(task_obj['task'])
    box.submit()

    try:
        description = browser.find_element(By.CLASS_NAME, 'description')
        task_obj['script'] = description.text
        task_obj['url'] = browser.current_url
        obj.append(task_obj)
        print(f'[{i}/{len(title_list)}]  {description.text}')

    except:
        error_id = {'len_id': i, 'task': task_obj['task']}
        not_page.append(error_id)
        box = browser.find_element(By.NAME, 'q')
        box.clear()

    time.sleep(1)

[0/499]  Semantic Segmentation is a computer vision task in which the goal is to categorize each pixel in an image into a class or object. The goal is to produce a dense pixel-wise segmentation map of an image, where each pixel is assigned to a specific class or object. Some example benchmarks for this task are Cityscapes, PASCAL VOC and ADE20K. Models are usually evaluated with the Mean Intersection-Over-Union (Mean IoU) and Pixel Accuracy metrics.
( Image credit: CSAILVision )
[1/499]  Object Detection is a computer vision task in which the goal is to detect and locate objects of interest in an image or video. The task involves identifying the position and boundaries of objects in an image, and classifying the objects into different categories.
The state-of-the-art methods can be categorized into two main types: one-stage methods and two stage-methods:
One-stage methods prioritize inference speed, and example models include YOLO, SSD and RetinaNet.
Two-stage methods prioritize detect

In [5]:
descriptions = pd.DataFrame(obj)
display(descriptions)
descriptions.to_csv('./description.csv', index=False)

,task,script,url
0,Semantic Segmentation,Semantic Segmentation is a computer vision tas...,https://paperswithcode.com/task/semantic-segme...
1,Object Detection,Object Detection is a computer vision task in ...,https://paperswithcode.com/task/object-detection
2,Image Classification,Image Classification is a fundamental task tha...,https://paperswithcode.com/task/image-classifi...
3,Pose Estimation,Pose Estimation is a computer vision task wher...,https://paperswithcode.com/task/pose-estimation
4,Face Recognition,Facial Recognition is the task of making a pos...,https://paperswithcode.com/task/face-recognition
...,...,...,...
494,Cell Segmentation,Cell Segmentation is a task of splitting a mic...,https://paperswithcode.com/task/cell-segmentation
495,Clean-label Backdoor Attack (0.05%),This task has no description! Would you like t...,https://paperswithcode.com/task/clean-label-ba...
496,Clothes Landmark Detection,This task has no description! Would you like t...,https://paperswithcode.com/task/clothes-landma...
497,Co-Salient Object Detection,Co-Salient Object Detection is a computational...,https://paperswithcode.com/task/co-saliency-de...


In [10]:
class Translator:

    def __init__(self):
        options = Options()
        options.add_argument('--headless')
        browser = webdriver.Chrome(options=options)
        browser.implicitly_wait(3)
        self._browser = browser

    def translate(self, text, dest='ja'):
        browser = self._browser

        # 翻訳したい文をURLに埋め込んでからアクセスする
        text_for_url = urllib.parse.quote_plus(text, safe='')
        url = "https://translate.google.co.jp/#en/ja/{0}".format(text_for_url)
        browser.get(url)

        # 数秒待機する
        wait_time = 2 + len(text) / 100
        time.sleep(wait_time)

        # 翻訳結果を抽出する
        ja = browser.find_element(By.CSS_SELECTOR ,"span[jsname='W297wb']")
        return ja.text

    def quit(self):
        self._browser.quit()


translator = Translator()

# ja = translator.translate('machine learning')
# print(ja) # => 機械学習

# ja = translator.translate('natural language processing')
# print(ja) # => 自然言語処理

In [11]:
description = pd.read_csv('./description.csv')
description.script = description.script.fillna('There is NaN')
description

,task,script,url
0,Semantic Segmentation,Semantic Segmentation is a computer vision tas...,https://paperswithcode.com/task/semantic-segme...
1,Object Detection,Object Detection is a computer vision task in ...,https://paperswithcode.com/task/object-detection
2,Image Classification,Image Classification is a fundamental task tha...,https://paperswithcode.com/task/image-classifi...
3,Pose Estimation,Pose Estimation is a computer vision task wher...,https://paperswithcode.com/task/pose-estimation
4,Face Recognition,Facial Recognition is the task of making a pos...,https://paperswithcode.com/task/face-recognition
...,...,...,...
494,Cell Segmentation,Cell Segmentation is a task of splitting a mic...,https://paperswithcode.com/task/cell-segmentation
495,Clean-label Backdoor Attack (0.05%),This task has no description! Would you like t...,https://paperswithcode.com/task/clean-label-ba...
496,Clothes Landmark Detection,This task has no description! Would you like t...,https://paperswithcode.com/task/clothes-landma...
497,Co-Salient Object Detection,Co-Salient Object Detection is a computational...,https://paperswithcode.com/task/co-saliency-de...


In [12]:
jpscript = []
error = []

In [13]:
for i in range(len(description)):
    try:
        dic = {}
        dic['idx'] = i
        dic['タスク'] = description.task[i] + '(' + translator.translate(description.task[i])    + ')'
        dic['概要'] = translator.translate(description.script[i])
        dic['url'] = description.url[i]
        print(f'({description.task[i]})' + dic['概要'] + '[' + str(i) + '/' + str(len(description)) + ']', flush=True)
        jpscript.append(dic)
    except:
        error.append({'idx': i, 'task': description.task[i]})
translator.quit()

(Semantic Segmentation)セマンティックセグメンテーションは、画像内の各ピクセルをクラスまたはオブジェクトに分類することを目標とするコンピュータービジョンタスクです。[0/499]
(Object Detection)オブジェクトの検出は、目標が画像またはビデオで関心のあるオブジェクトを検出して見つけることであるコンピュータービジョンタスクです。[1/499]
(Image Classification)画像分類は、画像全体を全体として理解しようとする基本的なタスクです。[2/499]
(Pose Estimation)ポーズ推定は、人またはオブジェクトの位置と方向を検出することを目標とするコンピュータービジョンタスクです。[3/499]
(Face Recognition)顔認識は、既存の顔のデータベースに対して写真またはビデオ画像で顔を肯定的に識別するタスクです。[4/499]
(Visual Question Answering (VQA))視覚的な質問応答（VQA）は、画像に関する質問への回答を含むコンピュータービジョンのタスクです。[5/499]
(Image Retrieval)画像検索は、特定のクエリ画像に似た大規模なデータベース内の画像を検索することを含むコンピュータービジョンタスクです。[6/499]
(Depth Estimation)深さの推定は、カメラに対する各ピクセルの距離を測定するタスクです。[7/499]
(Instance Segmentation)インスタンスセグメンテーションは、各オブジェクトの境界を検出したり、各オブジェクトに一意のラベルを割り当てるなど、画像内の個々のオブジェクトを識別および分離することを含むコンピュータービジョンタスクです。[8/499]
(Image Generation)ドラゴン[9/499]
(Question Answering)質問の回答は、質問に答えるタスク（通常は読解質問の質問）ですが、提供されたコンテキストに基づいて回答できない質問が提示された場合に棄権します。[10/499]
(Optical Character Recognition (OCR))光学キャラクター認識または光学文字リーダー（OCR）は、スキャンされたドキュメント、ドキュメントの写真、シーンフォト（例えば

In [14]:
name = './jp_image.csv'
ename = './jp_image_error.csv'
jp_summary = pd.DataFrame(jpscript)
jp_summary.to_csv(name, index=False)